In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tifffile import imread
from skimage.morphology import binary_dilation

np.set_printoptions(precision = 2)
np.set_printoptions(suppress  = True)

In [2]:
def normalize(img):
    img_copy  = img.copy()
    img_copy  = img_copy.astype('float64') 
    img_min   = np.min(img_copy)
    img_max   = np.max(img_copy)
    img_copy  = (img_copy - img_min)/(img_max - img_min)
    return img_copy

def create_circle_filter(N1, N2, R):
    n1   = np.linspace(-N1/2, N1/2, N1, dtype = 'float64')
    n2   = np.linspace(-N2/2, N2/2, N2, dtype = 'float64')
    n1, n2 = np.meshgrid(n1, n2)
    img  = (np.sqrt((n1/R)**2 + (n2/R)**2) <= 1)
    img  = img.astype('float64')
    return img

def convert_8bits(img):
    img_copy  = normalize(img)
    img_copy  = np.round(255*img_copy).astype('uint8')
    return img_copy

In [3]:
img  = imread('../data/multi_recorte_1.tif')
mask = mpimg.imread('../data/multi_recorte_1.png')[:,:,0]
mask = convert_8bits(mask)

images_path = 'images/'
masks_path  = 'masks/'
edges_path  = 'edges/'

In [6]:
krnl    = create_circle_filter(15, 15, 5)
mask_2  = binary_dilation(mask, krnl)
mask_2  = convert_8bits(mask_2)
edge    = mask_2 - mask

In [7]:
# Parameters
Nx, Ny, Nz = img.shape
Nw   = 256
step = round(Nw/4)

# Index images
indx_x = np.unique(np.concatenate(([np.arange(0, Nx-Nw, step)], [np.array([Nx-Nw])]), axis = 1))
indx_y = np.unique(np.concatenate(([np.arange(0, Ny-Nw, step)], [np.array([Ny-Nw])]), axis = 1))

# Total images
N      = indx_x.size*indx_y.size

# Cut images and save them
cntr   = 0
X_test = np.zeros((N, Nw, Nw, Nz))
for i in indx_x:
    for j in indx_y:
        cntr  += 1
        i_idx = np.arange(i, i+Nw)
        j_idx = np.arange(j, j+Nw)
        img_block = img[i_idx[:, None], j_idx[None, :], :]
        img_block = convert_8bits(img_block)
        mask_block = mask[i_idx[:, None], j_idx[None, :]]
        edge_block = edge[i_idx[:, None], j_idx[None, :]]
        plt.imsave('../' + images_path + str(cntr) +'.png', img_block) 
        plt.imsave('../' + masks_path + str(cntr) +'.png', mask_block, cmap = 'gray') 
        plt.imsave('../' + edges_path + str(cntr) +'.png', edge_block, cmap = 'gray')